In [2]:
pwd

'C:\\Users\\Baron\\Desktop\\EE_258_Repo\\EE_258\\ML_PATH_EE258\\EE258_env\\EE-210'

### Calculator For FFT

In [5]:
import numpy as np


import numpy as np

def twiddle(N, k):
    """
    Compute the twiddle factor W_N^k = exp(-j * 2π * k / N)
    """
    return np.exp(-2j * np.pi * k / N)

def fft_radix2(x):
    """
    Recursive radix-2 DIT FFT implementation using twiddle() helper.
    Input:  x (list or np.array) of length N = 2^n
    Output: FFT of x (as a list of complex numbers)
    """
    N = len(x)
    if N <= 1:
        return x
    if N % 2 != 0:
        raise ValueError("Input length must be a power of 2")

    # Divide: even and odd index samples
    even = fft_radix2(x[0::2])
    odd  = fft_radix2(x[1::2])

    # Combine using twiddle factors
    T = [twiddle(N, k) * odd[k] for k in range(N // 2)]
    
    return [even[k] + T[k] for k in range(N // 2)] + \
           [even[k] - T[k] for k in range(N // 2)]

In [9]:
x = [1, 2, 3, 4]  # Input signal (length must be a power of 2)
X = fft_radix2(x)

print("FFT result:")
for i, val in enumerate(X):
    print(f"A[{i}] = {val}")


FFT result:
A[0] = (10+0j)
A[1] = (-2+2j)
A[2] = (-2+0j)
A[3] = (-1.9999999999999998-2j)


In [19]:
import sympy as sp

def sym_twiddle(N, k):
    """Symbolic twiddle factor W_N^k = exp(-j*2π*k/N)"""
    j = sp.I
    return sp.exp(-2 * sp.pi * j * k / N)

def fft_radix2_sym(x):
    N = len(x)
    assert (N & (N - 1) == 0) and N != 0, "Length must be a power of 2"
    if N == 1:
        return x

    even = fft_radix2_sym(x[::2])
    odd = fft_radix2_sym(x[1::2])
    T = [sym_twiddle(N, k) * odd[k] for k in range(N // 2)]

    return [even[k] + T[k] for k in range(N // 2)] + \
           [even[k] - T[k] for k in range(N // 2)]

In [20]:
a, b, c, d = sp.symbols('a b c d')
x = [a, b, c, d]

X = fft_radix2_sym(x)

for i, expr in enumerate(X):
    print(f'X[{i}] = {sp.simplify(expr)}')

X[0] = a + b + c + d
X[1] = a - c + I*(-b + d)
X[2] = a - b + c - d
X[3] = a - c + I*(b - d)


In [21]:
import sympy as sp

def sym_twiddle(N, k):
    j = sp.I
    return sp.exp(-2 * sp.pi * j * k / N)

def fft_radix2_sym_verbose(x, level=0):
    N = len(x)
    indent = "  " * level

    if N == 1:
        return x

    # Split even and odd
    even = fft_radix2_sym_verbose(x[::2], level + 1)
    odd = fft_radix2_sym_verbose(x[1::2], level + 1)

    # Compute and print butterfly combinations
    X = [0] * N
    for k in range(N // 2):
        W = sym_twiddle(N, k)
        t = W * odd[k]
        X[k] = even[k] + t
        X[k + N // 2] = even[k] - t

        print(f"{indent}Stage (N={N}) - Butterfly k={k}:")
        print(f"{indent}  W_{N}^{k} = {sp.simplify(W)}")
        print(f"{indent}  T = {W} * {odd[k]} = {sp.simplify(t)}")
        print(f"{indent}  X[{k}] = {even[k]} + {sp.simplify(t)} = {sp.simplify(X[k])}")
        print(f"{indent}  X[{k + N//2}] = {even[k]} - {sp.simplify(t)} = {sp.simplify(X[k + N//2])}\n")

    return X

In [22]:
x0, x1, x2, x3 = sp.symbols('x0 x1 x2 x3')
x = [x0, x1, x2, x3]
X = fft_radix2_sym_verbose(x)

print("Final FFT result:")
for i, xi in enumerate(X):
    print(f"X[{i}] = {sp.simplify(xi)}")


  Stage (N=2) - Butterfly k=0:
    W_2^0 = 1
    T = 1 * x2 = x2
    X[0] = x0 + x2 = x0 + x2
    X[1] = x0 - x2 = x0 - x2

  Stage (N=2) - Butterfly k=0:
    W_2^0 = 1
    T = 1 * x3 = x3
    X[0] = x1 + x3 = x1 + x3
    X[1] = x1 - x3 = x1 - x3

Stage (N=4) - Butterfly k=0:
  W_4^0 = 1
  T = 1 * x1 + x3 = x1 + x3
  X[0] = x0 + x2 + x1 + x3 = x0 + x1 + x2 + x3
  X[2] = x0 + x2 - x1 + x3 = x0 - x1 + x2 - x3

Stage (N=4) - Butterfly k=1:
  W_4^1 = -I
  T = -I * x1 - x3 = I*(-x1 + x3)
  X[1] = x0 - x2 + I*(-x1 + x3) = x0 - x2 + I*(-x1 + x3)
  X[3] = x0 - x2 - I*(-x1 + x3) = x0 - x2 + I*(x1 - x3)

Final FFT result:
X[0] = x0 + x1 + x2 + x3
X[1] = x0 - x2 + I*(-x1 + x3)
X[2] = x0 - x1 + x2 - x3
X[3] = x0 - x2 + I*(x1 - x3)
